## 앙상블(Ensemble)
머신러닝 앙상블이란 **여러개의 머신러닝 모델을 이용해 최적의 답을 찾아내는 기법**이다.
- 여러 모델을 이용하여 데이터를 학습하고 모든 모델의 예측 결과를 평균하여 예측

**앙상블 기법의 종류**
- 보팅_Voting : 투표를 통한 결과 도출
- 배깅_Bagging : 샘플 중복 생성을 통한 결과 도출
- 부스팅_Boosting : 이전 오차를 보완하면서 가중치 부여
- 스태킹_Stacking : 여러 모델을 기반으로 예측된 결과를 통해 메타모델(예측모델)이 최종 예측 

In [2]:
import pandas as pd
import numpy as np

np.set_printoptions(suppress=True)

In [ ]:
# 데이터 셋 로드

from sklearn.datasets import fetch_openml

data = fetch_openml(name="boston", version=1, as_frame=True)

c:\ProgramData\anaconda3\envs\conda_env\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


#### 컬럼 소개
속성 수 : 13
- CRIM: 범죄율
- ZN: 25,000 평방 피트 당 주거용 토지의 비율
- INDUS: 비소매(non-retail) 비즈니스 면적 비율
- CHAS: 찰스 강 더미 변수 (통로가 하천을 향하면 1; 그렇지 않으면 0)
- NOX: 산화 질소 농도 (천만 분의 1)
- RM:주거 당 평균 객실 수
- AGE: 1940 년 이전에 건축된 자가 소유 점유 비율
- DIS: 5 개의 보스턴 고용 센터까지의 가중 거리
- RAD: 고속도로 접근성 지수
- TAX: 10,000 달러 당 전체 가치 재산 세율
- PTRATIO 도시 별 학생-교사 비율
- B: 1000 (Bk-0.63) ^ 2 여기서 Bk는 도시 별 검정 비율입니다.
- LSTAT: 인구의 낮은 지위
- MEDV: 자가 주택의 중앙값 (1,000 달러 단위)

In [10]:
print(dir(data))

['DESCR', 'categories', 'data', 'details', 'feature_names', 'frame', 'target', 'target_names', 'url']


In [13]:
data.target

0      24.0
1      21.6
2      34.7
3      33.4
4      36.2
       ... 
501    22.4
502    20.6
503    23.9
504    22.0
505    11.9
Name: MEDV, Length: 506, dtype: float64

In [6]:
print(data.data)

        CRIM    ZN  INDUS CHAS    NOX     RM   AGE     DIS RAD    TAX  \
0    0.00632  18.0   2.31    0  0.538  6.575  65.2  4.0900   1  296.0   
1    0.02731   0.0   7.07    0  0.469  6.421  78.9  4.9671   2  242.0   
2    0.02729   0.0   7.07    0  0.469  7.185  61.1  4.9671   2  242.0   
3    0.03237   0.0   2.18    0  0.458  6.998  45.8  6.0622   3  222.0   
4    0.06905   0.0   2.18    0  0.458  7.147  54.2  6.0622   3  222.0   
..       ...   ...    ...  ...    ...    ...   ...     ...  ..    ...   
501  0.06263   0.0  11.93    0  0.573  6.593  69.1  2.4786   1  273.0   
502  0.04527   0.0  11.93    0  0.573  6.120  76.7  2.2875   1  273.0   
503  0.06076   0.0  11.93    0  0.573  6.976  91.0  2.1675   1  273.0   
504  0.10959   0.0  11.93    0  0.573  6.794  89.3  2.3889   1  273.0   
505  0.04741   0.0  11.93    0  0.573  6.030  80.8  2.5050   1  273.0   

     PTRATIO       B  LSTAT  
0       15.3  396.90   4.98  
1       17.8  396.90   9.14  
2       17.8  392.83   4.03  
3  

In [12]:
df = pd.DataFrame(data=data.data, columns=data.feature_names)

In [ ]:
df["MEDV"] = data.target
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [ ]:
from sklearn.model_selection import train_test_split

# train/ test 분할
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("MEDV", axis=1), df["MEDV"], random_state=44
)

In [20]:
X_train.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
135,0.55778,0.0,21.89,0,0.624,6.335,98.2,2.1107,4,437.0,21.2,394.67,16.96
480,5.82401,0.0,18.10,0,0.532,6.242,64.7,3.4242,24,666.0,20.2,396.90,10.74
79,0.08387,0.0,12.83,0,0.437,5.874,36.6,4.5026,5,398.0,18.7,396.06,9.10
137,0.35233,0.0,21.89,0,0.624,6.454,98.4,1.8498,4,437.0,21.2,394.08,14.59
176,0.07022,0.0,4.05,0,0.510,6.020,47.2,3.5549,5,296.0,16.6,393.23,10.11


## 평가지표
- MSE   [Mean Squared Error]
 > 실제값과 예측값의 차의 제곱값의 평균
- MAE   [Mean Absolute Error]
 > 실제값과 예측값의 차에 절대값의 평균
- RMSE  [Rooted Mean Squared Error]
 > 실제값과 예측값의 차이에 대한 평균에 루트를 씌운 값

### 모델별 성능 확인을 위한 함수

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns



my_predictions = {}


colors = [
    "r",
    "c",
    "m",

    "y",
    "k",

    "khaki",
    "teal",

    "orchid",
    "sandybrown",
    "greenyellow",
    "dodgerblue",
    "deepskyblue",
    "rosybrown",
    "firebrick",
    "deeppink",
    "crimson",
    "salmon",

    "darkred",
    "olivedrab",
    "olive",
    "forestgreen",

    "royalblue",
    "indigo",
    "navy",
    "mediumpurple",

    "chocolate",
    "gold",
    "darkorange",
    "seagreen",
    "turquoise",
    "steelblue",

    "slategray",
    "peru",

    "midnightblue",
    "slateblue",

    "dimgray",
    "cadetblue",
    "tomato",

]



def plot_predictions(name_, pred, actual):

    df = pd.DataFrame({"prediction": pred, "actual": y_test})

    df = df.sort_values(by="actual").reset_index(drop=True)

    plt.figure(figsize=(12, 9))

    plt.scatter(df.index, df["prediction"], marker="x", color="r")
    plt.scatter(df.index, df["actural"], alpha=0.7, marker="o", color="black")
    plt.title(name_, fontsize=15)
    plt.legend(["prediction", "actual"], fontsize=12)
    plt.show()


def mes_eval(name_, pred, actual):
    global predictions
    global colors

    plot_predictions(name_, pred, actual)

    mse = mean_squared_error(pred, actual)
    my_predictions[name_] = mse

    y_value = sorted(my_predictions.items(), key=lambda x: -x[1])

    df = pd.DataFrame(y_value, colums=["model", "mse"])
    print(df)
    min_ = df["mse"].min() - 10
    max_ = df["mse"].max() + 10
    length = len(df)

    plt.figure(figsize=(10, length))
    ax = plt.subplot()
    ax.set_yticks(np.arange(len(df)))
    ax.set_yticklabels(df["model"], fontsize=15)
    bars = ax.barh(np.arange(len(df)), df["mse"])

    for i, v in enumerate(df["mse"]):
        idx = np.random.choice(len(colors))
        bars[i].set_color(colors[idx])
        ax.text(v + 2, i, str(round(v, 3)), color="k", fontsize=15, fontwight="bold")

    plt.title("MSE Error", fontsize=18)
    plt.xlim(min_, max_)

    plt.show()


def remove_model(name_):
    global my_predictions
    try:
        del my_predictions[name_]
    except KeyError:
        return False
    return True